In [1]:
import pandas as pd
gift = pd.read_csv('GIFT_REP_Northwestern_v2.csv',low_memory=False)

In [2]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [3]:
gifttest=gift
len(gifttest)

519793

In [4]:
gifttest

,Key Indicator,Constituent ID,Gift Date,Gift Amount,Appeal Split Amount,Gift Type,Category,Campaign ID,Campaign Description,Fund ID,Fund Description,Appeal ID,Appeal Description,Luminate Online Gift ID,NetCommunity Online Gift ID,Gift Code,OPX Provider Description
0,Individual,1026027.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98KR,98 April Renewal,NaN,NaN,NaN,NaN
1,Individual,1041282.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98LR,98 May Renewal,NaN,NaN,NaN,NaN
2,Individual,1041328.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98JR,98 March Renewal,NaN,NaN,NaN,NaN
3,Individual,1111161.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98LR,98 May Renewal,NaN,NaN,NaN,NaN
4,Individual,1113516.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,Direct Mail - Old,98LR,98 May Renewal,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519788,Individual,1247705.0,6/30/2023,$60.00,$60.00,Cash,Workplace Giving,4009 PYDEDLCF,Payroll Deductions LCF - 4009,Payroll LCF Employee,Payroll Deductions - LCF Employees,LCFEMPGIV23,LCF Employee Giving Campaign FY23,NaN,NaN,NaN,NaN
519789,Individual,1049457.0,6/30/2023,$100.00,$100.00,Cash,Individual Giving,4001 INDGIV,Individual Giving - 4001,700 FUND,General Contribution,NOV23R,November 2022 Appeal,NaN,NaN,NaN,NaN
519790,Individual,1248910.0,6/30/2023,$45.00,$45.00,Cash,Workplace Giving,4009 PYDEDLCF,Payroll Deductions LCF - 4009,Payroll LCF Employee,Payroll Deductions - LCF Employees,LCFEMPGIV23,LCF Employee Giving Campaign FY23,NaN,NaN,NaN,NaN
519791,Organization,1269332.0,6/30/2023,$75.00,$75.00,MG Pledge,Individual Giving,4005 C&FCONT,Corporations & Foundations Contributions - 4005,700 FUND,General Contribution,JUN23R,June 2023 Appeal,NaN,NaN,NaN,NaN


# Cleaning: String Strip
- combine 'Appeal Description' and 'Campaign Description' to a new column called 'concatenated'
- If NAs present in the 'concatenated' column, fill in with values from 'Fund Description'
- strip string by removing punctuations, duplicate words, and unnecessary date info

In [5]:
#concate 'Appeal Description' compaign description
gifttest['concatenated']=gifttest['Appeal Description']+' '+gifttest['Campaign Description']
#fillna fund description
gifttest['concatenated'].fillna(gifttest['Fund Description'], inplace=True)
# gifttest = gifttest.dropna(subset=['concatenated'])
gifttest['concatenated']=gifttest['concatenated'].str.lower()
#strip
# Define regular expression patterns
pattern_numbers = r'\d+'
pattern_months = r'\b(?:january|february|march|april|may|june|july|august|september|october|november|december)\b'
pattern_punctuation = r'[^\w\s]'
pattern_duplicated_words = r'\b(\w+)\b(?=.*\b\1\b)'
def duplicate(text):
    words = text.split()
    unique_words = list(dict.fromkeys(words))
    unique_text = ' '.join(unique_words)
    return unique_text

gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_numbers, '', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_months, '', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_punctuation, ' ', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern_duplicated_words, r'\1', regex=True)
gifttest['concatenated'] = gifttest['concatenated'].apply(lambda x: duplicate(x))

In [6]:
len(gifttest['concatenated'])

519793

# EDA: Event Words

### Use Counter to see the top frequent words 

In [7]:
from collections import Counter
words = ' '.join(gifttest['concatenated']).split()
word_count = Counter(words)
common_keywords = [word for word, count in word_count.most_common() if count > 2]


#drop stopwords
word_count.most_common()[:20]

[('mail', 212132),
 ('direct', 211926),
 ('old', 181770),
 ('fy', 127893),
 ('giving', 90170),
 ('mo', 79146),
 ('x', 73194),
 ('renewal', 55380),
 ('general', 53055),
 ('annual', 52400),
 ('appeal', 48485),
 ('budget', 44705),
 ('operating', 43037),
 ('newsletter', 41724),
 ('of', 31315),
 ('circle', 27094),
 ('love', 26585),
 ('campaign', 24360),
 ('end', 21412),
 ('individual', 18763)]

In [8]:
import re
stopwordlist = ['fy', 'mo', 'x', 'end']

# Create a regular expression pattern to match stopwords
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in stopwordlist) + r')\b'

# Remove stopwords from the 'concatenated' column
gifttest['concatenated'] = gifttest['concatenated'].str.replace(pattern, '', regex=True)
gifttest['concatenated'] 

0                                          direct mail old
1                                          direct mail old
2                                          direct mail old
3                                          direct mail old
4                                          direct mail old
                                ...                       
519788    lcf employee giving campaign  payroll deductions
519789                            appeal individual giving
519790    lcf employee giving campaign  payroll deductions
519791       appeal corporations foundations contributions
519792    lcf employee giving campaign  payroll deductions
Name: concatenated, Length: 519793, dtype: object

### Use Document-term matrix to see the top frequent words
- set ngram to be 2-4, since the event name may contain several words (one word doesn't make sense)
- find the combination with the highest occurrences: up to the top 250

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(ngram_range=(2, 4))
X = cv.fit_transform(gifttest['concatenated'])

# Convert the result to a DataFrame for analysis
count_matrix = pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())
count_matrix[:10]

,abbott work,abbott work place,abbott work place giving,abbvie work,abbvie work place,abbvie work place giving,abcf campaign,abcf campaign acquisition,above general,above general contr,...,ypg food frenzy,ypg food frenzy general,ypg food frenzy special,ypg little,ypg little city,ypg little city on,zips acquisitions,zurich employee,zurich employee giving,zurich employee giving workplace
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
combination_counts = count_matrix.sum()

# Find the combination with the highest occurrences
max_combination = combination_counts.idxmax()
max_occurrences = combination_counts[max_combination]
print(combination_counts.sort_values(ascending=False)[:50])

direct mail                            211317
direct mail old                        181073
mail old                               181073
general operating                       43037
operating budget                        43035
general operating budget                43035
annual giving                           41202
circle of love                          26582
of love                                 26582
circle of                               26582
individual giving                       17322
employee giving                         17191
work place                              16083
special events                          13733
love annual giving                      11421
of love annual giving                   11421
of love annual                          11421
circle of love annual                   11421
love annual                             11421
year appeal                             11261
newsletter general operating            10884
newsletter general operating budge

### Manually create a new events_list 
- refer to events in orginal .csv file
- based on former top frequent events (eyeball)

In [11]:
events_list = ['general operating budget','direct mail','circle of love','annual giving','newsletter','pledge','annual dinner','little city invitational','santa stickers','smiles campaign','holiday mailing','holiday cards','teaser','renewal','golf','teddy bear','verne carson letter','face labels','calendar','donor campaign','university of illinois work place giving','book sale','parent']

In [19]:
len(events_list)

23

### Iterate over each row in the .csv file
- create a empty list for each row
- if one (or more) of the events in the events_list is found in each row's 'concatenated' column accordingly, append that event to the list (one row can have more than one event/way of donation)

In [12]:
# Iterate through each row of the 'gifttest' DataFrame
for index, row in gifttest.iterrows():
    # Create a list to store matching events for the current row
    matching_events_list = []
    
    # Iterate through each event in 'events_list'
    for event in events_list:
        # Check if the lowercase version of 'concatenated' contains the lowercase version of the current event
        if row['concatenated'].lower().find(event.lower()) != -1:
            # If the event is found, append it to the matching events list
            matching_events_list.append(event)

    # Assign the matching events list to the 'matching_event' column for the current row
    gifttest.at[index, 'matching_event'] = matching_events_list

/var/folders/g5/t835zv0x7sb7qvk1j7k7t9y40000gp/T/ipykernel_72999/4123412955.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'direct mail' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  gifttest.at[index, 'matching_event'] = matching_events_list


In [13]:
gifttest['matching_event'].value_counts()

matching_event
[direct mail]                                           190880
[]                                                      108417
[renewal]                                                45207
[general operating budget]                               22241
[annual giving]                                          16872
[newsletter]                                             13511
[circle of love, annual giving]                          11421
[general operating budget, newsletter]                   10890
[pledge]                                                 10825
[direct mail, newsletter]                                 9271
[annual giving, newsletter]                               8026
[parent]                                                  7819
[circle of love]                                          5476
[general operating budget, circle of love]                5237
[annual dinner]                                           4691
[annual giving, holiday mailing]        

### Replace empty list [] with NA values

In [14]:
empty_list_rows = gifttest[gifttest['matching_event'].apply(lambda x: x == [])]
import numpy as np

# Replace empty lists with NaN in the 'matching_event' column
gifttest['matching_event'] = gifttest['matching_event'].apply(lambda x: np.nan if x == [] else x)

### Create a subcategory for event called 'newsletter'
- it has a four-season series: 'spring', 'summer', 'fall', 'winter'

In [15]:
# Define a function to check conditions and return the matched season
def get_season(row):
    seasons = ['spring', 'summer', 'fall', 'winter']
    for season in seasons:
        if 'newsletter' in row['concatenated'].lower() and season in row['concatenated'].lower():
            return season

# Apply the function to create a new column 'newsletter_sub'
gifttest['newsletter_sub'] = gifttest.apply(get_season, axis=1)

# Display the updated DataFrame
gifttest['newsletter_sub'].value_counts()


newsletter_sub
fall      9415
winter    6194
spring    5137
summer    4918
Name: count, dtype: int64

# Create Dummy Columns

In [16]:

# Explode the lists in the 'matching_event' column to separate rows
gifttest_exploded = gifttest['matching_event'].explode('matching_event')

# Create dummy columns for each event
dummy_columns = pd.get_dummies(gifttest_exploded, prefix='event')
dummy_columns = dummy_columns.astype(int)  # or dummy_columns = dummy_columns * 1

# Concatenate the dummy columns with the original dataframe
gifttest = pd.concat([gifttest, dummy_columns], axis=1)

# Display the resulting dataframe
gifttest.head(10)

,Key Indicator,Constituent ID,Gift Date,Gift Amount,Appeal Split Amount,Gift Type,Category,Campaign ID,Campaign Description,Fund ID,...,event_newsletter,event_parent,event_pledge,event_renewal,event_santa stickers,event_smiles campaign,event_teaser,event_teddy bear,event_university of illinois work place giving,event_verne carson letter
0,Individual,1026027.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
1,Individual,1041282.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
2,Individual,1041328.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
3,Individual,1111161.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
4,Individual,1113516.0,7/1/1998,$5.00,$5.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
5,Individual,1033818.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
6,Individual,1027399.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
7,Individual,1043888.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
8,Individual,1094326.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0
9,Individual,1106432.0,7/1/1998,$10.00,$10.00,Cash,Individual Giving,NaN,NaN,DM,...,0,0,0,0,0,0,0,0,0,0


In [17]:
gifttest.to_csv('merge_file.csv', index=False)

# One-hot encoding
1 - Yes

0 - No

### Way of Donation: Direct Mail

In [21]:
gifttest['Direct Mail'] = gifttest['Fund Description'].apply(lambda x: 1 if pd.notna(x) and 'Direct Mail' in x else 0)
gifttest['Direct Mail'].value_counts()

Direct Mail
1    417896
0    180177
Name: count, dtype: int64

Almost 80% of the data are from the way of 'Direct Mail' which may due to the lack of record for events/appeals in the early decades. (That's why we substitute the missing values with 'Direct Mail')

### Loyalty: Renewal


In [23]:
gifttest['Renewal'] = gifttest['concatenated'].apply(lambda x: 1 if pd.notna(x) and 'renewal' in x else 0)
gifttest['Renewal'].value_counts()

Renewal
0    542693
1     55380
Name: count, dtype: int64

Only about 10% of the data are from the 'Renewal'.

### Format the 'Gift Amount' column

In [24]:
gifttest['Gift Amount'] = gifttest['Gift Amount'].str.replace('$', '').str.replace(',', '').str.strip()

In [31]:
gifttest['Gift Amount'] = pd.to_numeric(gifttest['Gift Amount'], errors='coerce')

### Total amount of donation in the file

In [32]:
total_gift_amount = gifttest['Gift Amount'].sum()
print(total_gift_amount)

110648616.72


# Sum over giftamount for each event category
- iterate over row
- find the matched event in the 'matching_event' column
- calculate the cumulative sum of each event and round to integer
- save into a dictionary called sumdict

In [33]:
sumdict = {}

for index, row in gifttest.iterrows():
    matching_events = row['matching_event']
    
    # Check if matching_events is iterable
    if not isinstance(matching_events, (list, tuple)):
        matching_events = [matching_events]

    for event in matching_events:
        if event not in sumdict:
            sumdict[event] = row['Gift Amount']
        else:
            sumdict[event] += row['Gift Amount']

sorted_sumdict = dict(sorted(sumdict.items(), key=lambda x: x[1], reverse=True))

# Round the final result to 2 decimal places
rounded_sumdict = {key: round(value, 0) for key, value in sorted_sumdict.items()}

Note here we may count the giftamount more than once if its list has more than one event. Say, if one row has an list of matching_event = ['direct mail', 'circle of love'], we will add its giftamount to both category. 

In [34]:
rounded_sumdict

{'direct mail': 12162844.0,
 nan: nan,
 'annual dinner': 8710156.0,
 'golf': 7910865.0,
 'little city invitational': 7517408.0,
 'general operating budget': 4516240.0,
 'annual giving': 3343338.0,
 'newsletter': 2065857.0,
 'parent': 1887711.0,
 'renewal': 1528069.0,
 'circle of love': 1214886.0,
 'book sale': 972638.0,
 'holiday mailing': 368919.0,
 'pledge': 353209.0,
 'holiday cards': 237509.0,
 'teaser': 136834.0,
 'donor campaign': 135125.0,
 'teddy bear': 118806.0,
 'verne carson letter': 99202.0,
 'santa stickers': 86194.0,
 'calendar': 74116.0,
 'smiles campaign': 8782.0,
 'face labels': 5141.0,
 'university of illinois work place giving': 3889.0}

### Percentage of donation through 'direct mail'

In [35]:
round(rounded_sumdict['direct mail'] / total_gift_amount *100,2)

10.99

### Percentage of donation through 'renewal'

In [36]:
round(rounded_sumdict['renewal'] / total_gift_amount * 100,2)

1.38

### Save the file for dashboard preparation

In [37]:
df = pd.DataFrame(list(rounded_sumdict.items()), columns=['Event', 'Rounded Value'])
df.to_csv('sumdict.csv')

# Count over each event category
- iterate over row
- find the matched event in the 'matching_event' column
- calculate the number of apperance of each event
- save into a dictionary called countdict

In [38]:
countdict = {}

for index, row in gifttest.iterrows():
    matching_events = row['matching_event']
    
    # Check if matching_events is iterable
    if not isinstance(matching_events, (list, tuple)):
        matching_events = [matching_events]

    for event in matching_events:
        if event not in countdict:
            countdict[event] = 1
        else:
            countdict[event] += 1

sorted_countdict = dict(sorted(countdict.items(), key=lambda x: x[1], reverse=True))

# Round the final result to 0 decimal places
rounded_countdict = {key: round(value, 0) for key, value in sorted_countdict.items()}

rounded_countdict

{'direct mail': 211317,
 nan: 186697,
 'renewal': 55380,
 'general operating budget': 43035,
 'newsletter': 41724,
 'annual giving': 41202,
 'circle of love': 26582,
 'pledge': 10847,
 'parent': 8608,
 'golf': 8434,
 'annual dinner': 5561,
 'little city invitational': 5494,
 'santa stickers': 5026,
 'holiday mailing': 4727,
 'teaser': 4500,
 'teddy bear': 4183,
 'calendar': 2814,
 'donor campaign': 2420,
 'verne carson letter': 2265,
 'university of illinois work place giving': 2131,
 'book sale': 2123,
 'holiday cards': 1104,
 'face labels': 154,
 'smiles campaign': 25}

### Save the file for dashboard preparation

In [39]:
df = pd.DataFrame(list(rounded_countdict.items()), columns=['Event', 'Count'])
df.to_csv('countdict.csv')

# Future discussion
- may drop NAN when presenting the results in dashboard (because we didn't cover all the events yet)

-> how to deal with much NAs?
- may use eyeballs to catch more events to decrease the number of NA values